In [24]:
import pandas as pd


# Updated data
updated_data = {
    'meteredValues': [0.0, 6.639, 0.0, 3.785, 3.075, 0.0, 6.647, 0.0, 0.0, 5.351, 0.0, 0.0, 1.722],
    'peakDemandTimes': [
        '2023-11-28 00:00:00', '2023-11-28 00:15:00', '2023-11-28 00:30:00', '2023-11-28 00:45:00',
        '2023-11-28 01:00:00', '2023-11-28 01:15:00', '2023-11-28 01:30:00',
        '2023-11-28 01:45:00', '2023-11-28 02:00:00', '2023-11-28 02:15:00',
        '2023-11-28 02:30:00', '2023-11-28 02:45:00', '2023-11-28 03:00:00',
    ]
}

df_updated = pd.DataFrame(updated_data)

# Convert 'peakDemandTimes' to datetime
df_updated['peakDemandTimes'] = pd.to_datetime(df_updated['peakDemandTimes'])

# Round 'peakDemandTimes' to the nearest hour
df_updated['peakDemandTimes'] = df_updated['peakDemandTimes'].dt.ceil('H')

# Group by hourly and sum 'meteredValues'
result_updated = df_updated.groupby(df_updated['peakDemandTimes'])['meteredValues'].sum()()

print(result_updated)

# Print peakDemandTimes, meteredValues, and result
for hour, metered_value in zip(df_updated['peakDemandTimes'], df_updated['meteredValues']):
    print(f"{hour.strftime('%Y-%m-%d %H:%M:%S')} | {metered_value} | {result_updated[hour]}")

      peakDemandTimes  meteredValues
0 2023-11-28 00:00:00          0.000
1 2023-11-28 01:00:00         13.499
2 2023-11-28 02:00:00          6.647
3 2023-11-28 03:00:00          7.073


KeyError: Timestamp('2023-11-28 00:00:00')

In [58]:

import sys 
sys.path.append('/home/pi/smart_plug/')
import os
import pandas as pd
import sqlite3
from datetime import datetime, timedelta
from database.db_manager import DatabaseManager
import datetime
 
from external_services.awattar_services import AwattarServices
from external_services.smartmeter_services import SmartMeterServices
from database.db_manager import DatabaseManager
from main_services.common_utils import common_utils 
 
import pandas as pd
import datetime


# Datetime
# Get the current date
current_date = str(datetime.date.today() - datetime.timedelta(days=1))

# Combine with the time '00:00:00'
combined_datetime = pd.to_datetime(current_date + ' 00:00:00')


# Default dataset
default_data = {'meteredValues': [0],
    'peakDemandTimes': [combined_datetime]}

print(default_data)


{'meteredValues': [0], 'peakDemandTimes': [Timestamp('2023-11-28 00:00:00')]}


In [ ]:


import sys 
sys.path.append('/home/pi/smart_plug/')
import os
import pandas as pd
import sqlite3
from datetime import datetime, timedelta
from database.db_manager import DatabaseManager
 

# Get the current datetime
current_datetime = datetime.now()

# Calculate a datetime that is 30 minutes ago
datetime_30_minutes_ago = current_datetime - timedelta(minutes=40)

# Calculate a datetime that is 1 hour in the future
datetime_1_hour_in_future = current_datetime + timedelta(hours=1)

print("Current Datetime:", current_datetime)
print("Datetime 30 Minutes Ago:", datetime_30_minutes_ago)
print("Datetime 1 Hour in Future:", datetime_1_hour_in_future)

db = DatabaseManager()
status1 = db.read_relaymode_temp(1)
print(status1)

status2 = db.read_relaymode_temp(2)
print(status2)


current_datetime = datetime.now() - timedelta(hours=24)
start_of_day = current_datetime.replace(hour=0, minute=0, second=0, microsecond=0)
end_of_day = current_datetime.replace(hour=23, minute=59, second=59, microsecond=99)

print(start_of_day.strftime('%Y-%m-%d %H:%M:%S.%f'))
print(end_of_day.strftime('%Y-%m-%d %H:%M:%S'))



In [ ]:


import sys 
sys.path.append('/home/pi/smart_plug/')
import os
import pandas as pd
import sqlite3
from datetime import datetime, timedelta
from database.db_manager import DatabaseManager
 

current_datetime = datetime.now() - timedelta(hours=24)
current_datetime = current_datetime.strftime('%Y-%m-%d %H:%M:%S.%f')

print(current_datetime)

In [ ]:

import sys 
sys.path.append('/home/pi/smart_plug/')
import os
from main_services.common_utils import common_utils 

test = 'dataset/'+common_utils.static_awattar_filename

print(test)

In [1]:
import sys

sys.path.append('/home/pi/smart_plug/')
import os
import schedule
import time
from PIL import Image
import io

import sqlite3
from sqlite3 import Error

from datetime import datetime

from external_services.awattar_services import AwattarServices
from pi_controller.relay_controller import RelayControl
from external_services.smartmeter_services import SmartMeterServices
from database.db_manager import DatabaseManager
from main_services.common_utils import common_utils 


from flask import Flask, jsonify, request, send_file

from PIL import Image
import io
import pandas as pd
import matplotlib.pyplot as plt

current_dir = os.getcwd()
print("Current working directory:", current_dir)

app = Flask(__name__)


# Helper function to convert data to image
def data_to_image(data):
    # Assuming data is a binary representation of an image
    image = Image.open(io.BytesIO(data))
    return image


def getLiveRelayStatus():
    try:
            # Connect to the SQLite database
            db = DatabaseManager()
            
            conn = sqlite3.connect("/home/pi/smart_plug/database/pythonsqlite.db")
            cursor = conn.cursor()

            df = pd.read_sql_query("SELECT start_timestamp, end_timestamp, marketprice, unit, relaynumber FROM automaterelay", conn)

            # Check if the DataFrame is not empty
            if not df.empty:
                # Plot the DataFrame as a table
                fig, ax = plt.subplots(figsize=(8, 6))
                ax.axis('tight')
                ax.axis('off')
                table = ax.table(cellText=df.values, colLabels=df.columns, cellLoc='center', loc='center')
                image_file = io.BytesIO()
                plt.savefig(image_file, format='png', bbox_inches='tight')
                image_file.seek(0)
                plt.close()
                return image_file
            return None
               
    except Exception as e:
        print(f"An error occurred: {e}")
        return jsonify({"status": "Error"}), 500

    finally:
        conn.close()

def get_data_as_image():
    image_data = getLiveRelayStatus()
    if image_data:
        return send_file(image_data, mimetype='image/png')
    return jsonify({"status": "No data found"}), 404

get_data_as_image()

Current working directory: /home/pi/smart_plug/python_files
init called
2.6.0


RuntimeError: Working outside of request context.

This typically means that you attempted to use functionality that needed
an active HTTP request. Consult the documentation on testing for
information about how to avoid this problem.